### 0. Ścieżki

In [1]:
### 0. Ścieżki do plików CSV

"""
Ustawia wspólny katalog na wszystkie pliki CSV z tego notatnika.

OUTPUT_CSV_DIR – katalog na wyniki pośrednie (kandydaci na dc:subject itd.).
"""

import os

OUTPUT_CSV_DIR = "outputs/csv"
os.makedirs(OUTPUT_CSV_DIR, exist_ok=True)

### 1. Importy i konfiguracja środowiska

In [2]:
"""
Importy i konfiguracja podstawowa dla CLIP zero-shot.

Ujednolicone z 03_clip_scene_classification.ipynb.
"""

from pathlib import Path
from datetime import datetime, timezone
import json

import numpy as np
import pandas as pd
from PIL import Image

import torch
import clip  # lub open_clip, zależnie od 03
import matplotlib.pyplot as plt

"""
Wybór urządzenia i konfiguracja precyzji obliczeń dla CLIP.

Preferencja: mps → cuda → cpu.
"""

def get_device() -> torch.device:
    if torch.backends.mps.is_available():
        return torch.device("mps")
    if torch.cuda.is_available():
        return torch.device("cuda")
    return torch.device("cpu")

device = get_device()
IMG_DTYPE = torch.float16  # spójnie z 03
print("Używane urządzenie:", device, "| dtype:", IMG_DTYPE)

Używane urządzenie: mps | dtype: torch.float16


### 2. Ścieżki + ładowanie modelu

In [3]:
"""
Ścieżki wejścia/wyjścia i ładowanie modelu CLIP dla zero-shot.

Spójne nazewnictwo z 03_clip_scene_classification.ipynb:
- PROJECT_ROOT
- IMAGE_ROOT
- OUTPUT_DIR
- OUTPUT_CSV_DIR
- CSV_PATH
- RUNLOG_PATH
"""

from pathlib import Path

PROJECT_ROOT = Path(".").resolve()

# katalog z obrazami do zero-shot (dostosuj do swojej struktury)
IMAGE_ROOT = PROJECT_ROOT / "inputs"

# katalog na wyniki zero-shot (inne artefakty, np. wykresy)
OUTPUT_DIR = PROJECT_ROOT / "outputs" / "clip_zero_shot"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# wspólny katalog na wszystkie CSV
OUTPUT_CSV_DIR = PROJECT_ROOT / "outputs" / "csv"
OUTPUT_CSV_DIR.mkdir(parents=True, exist_ok=True)

# plik CSV z wynikami zero-shot (kandydaci na dc:subject)
CSV_PATH = OUTPUT_CSV_DIR / "clip_zero_shot_subjects.csv"

RUNLOG_PATH = PROJECT_ROOT / "logs" / "runlog.jsonl"
RUNLOG_PATH.parent.mkdir(parents=True, exist_ok=True)

# ładowanie modelu CLIP (jak w 03)
clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)
clip_model = clip_model.to(device=device, dtype=IMG_DTYPE).eval()

print("Załadowano model CLIP ViT-B-32")
print("PROJECT_ROOT:", PROJECT_ROOT)
print("IMAGE_ROOT:", IMAGE_ROOT)
print("Istnieje?", IMAGE_ROOT.exists())

100%|███████████████████████████████████████| 338M/338M [00:43<00:00, 8.17MiB/s]


Załadowano model CLIP ViT-B-32
PROJECT_ROOT: /Users/olga/MetaLogic
IMAGE_ROOT: /Users/olga/MetaLogic/inputs
Istnieje? True


### 3. Funkcje pomocnicze (wczytywanie obrazu)

In [4]:
"""
Funkcje pomocnicze do pracy z obrazami dla zero-shot CLIP.

Spójne z 03_clip_scene_classification.ipynb.
"""

def load_image(path: Path) -> Image.Image:
    """
    Wczytuje obraz z dysku jako RGB (bez zmiany kontrastu/jasności).

    path – pełna ścieżka do pliku obrazu.
    """
    img = Image.open(path)
    if img.mode != "RGB":
        img = img.convert("RGB")
    return img

### 4. Definicja promptów czasowych i etykiet przedziałów

In [5]:
"""
Definicja promptów zero-shot dla przedziałów czasowych (okresy historyczne).

Prompty w TEXT_PROMPTS opisują semantycznie trzy zakresy:
- "do 1944"
- "PRL 1945–1989"
- "po 1990"

Słownik PROMPT_TO_PERIOD mapuje pełen tekst promptu
na nazwę przedziału czasowego (etykietę wynikową).
"""

LABELS = ["do 1944", "PRL 1945–1989", "po 1990"]

TEXT_PROMPTS = [
    # A) Do 1944
    "photograph taken in Poland up to 1944; interwar or older clothing; coats, hats, uniforms; cobblestone streets; prewar tenement houses; horse carriages or very old cars; hand-painted shop signs; art deco typography; sepia or black and white style",

    # B) PRL 1945–1989
    "photo from the Polish People's Republic (1945–1989), PRL; prefab panel blocks, RUCH kiosk, neon signs, 'Społem' or 'Pewex' stores; queues and everyday street scenes; 1970s–1980s clothing with shaggy hairstyles, thick-rimmed glasses, polyester suits; Fiat 126p, Polonez, Żuk or Nysa vans; community theater or amateur performances; socialist-era typography and posters",

    # C) Po 1990
    "photograph in Poland after 1990; modern ads and global brand logos; PVC banners, colorful shop signs; street trade and open markets; modern cars after 2005; sportswear with visible logos; smartphones, glass office buildings, shopping malls, renovated tenement houses",
]

PROMPT_TO_PERIOD = dict(zip(TEXT_PROMPTS, LABELS))

### 5. Obliczanie score_CLIP dla wszystkich obrazów (zero-shot: przedziały czasowe)

In [6]:
"""
Obliczanie score_CLIP dla wszystkich obrazów dla trzech przedziałów czasowych.

Wejście:
- IMAGE_ROOT – katalog wejściowy z obrazami
- TEXT_PROMPTS – lista promptów opisujących okresy historyczne
- PROMPT_TO_PERIOD – mapowanie full_prompt → etykieta okresu

Wyjście:
- df_periods z kolumnami:
  'file_path', 'prompt_en', 'period_label', 'score_period'
"""

# lista obsługiwanych rozszerzeń
image_paths = (
    sorted(IMAGE_ROOT.rglob("*.jpg"))
    + sorted(IMAGE_ROOT.rglob("*.jpeg"))
    + sorted(IMAGE_ROOT.rglob("*.png"))
    + sorted(IMAGE_ROOT.rglob("*.tif"))
)

print("Liczba obrazów do zero-shot:", len(image_paths))

records = []

with torch.no_grad():
    # tokenizacja i embedding tekstów (prompty okresów)
    # truncate=True – pozwala na przycięcie zbyt długich promptów do context_length
    text_tokens = clip.tokenize(TEXT_PROMPTS, truncate=True).to(device)
    text_features = clip_model.encode_text(text_tokens)
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)

    for i, path in enumerate(image_paths, start=1):
        print(f"[{i}/{len(image_paths)}] {path.name}")

        img = load_image(path)
        image_input = clip_preprocess(img).unsqueeze(0).to(device, dtype=IMG_DTYPE)

        # embedding obrazu
        image_features = clip_model.encode_image(image_input)
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)

        # podobieństwo obraz–tekst
        logits = (100.0 * image_features @ text_features.T).squeeze(0)
        probs = logits.softmax(dim=-1).cpu().numpy()

        # wybór najlepszego promptu = najlepszy okres
        best_idx = int(np.argmax(probs))
        best_prompt = TEXT_PROMPTS[best_idx]
        best_score = float(probs[best_idx])
        best_period = PROMPT_TO_PERIOD[best_prompt]

        records.append(
            {
                "file_path": str(path.relative_to(PROJECT_ROOT)),
                "prompt_en": best_prompt,
                "period_label": best_period,
                "score_period": best_score,
            }
        )

df_periods = pd.DataFrame(records)
df_periods.head()

Liczba obrazów do zero-shot: 133
[1/133] 0000.jpg
[2/133] 000202.jpg
[3/133] 000238.jpg
[4/133] 0003.jpg
[5/133] 000345.jpg
[6/133] 000346.jpg
[7/133] 000388.jpg
[8/133] 000391.jpg
[9/133] 0004.jpg
[10/133] 000406.jpg
[11/133] 0006 2.jpg
[12/133] 0006.jpg
[13/133] 0009.jpg
[14/133] 0022.jpg
[15/133] 0034.jpg
[16/133] 0043.jpg
[17/133] 0044.jpg
[18/133] 0074.jpg
[19/133] 0075.jpg
[20/133] 0077.jpg
[21/133] 0079.jpg
[22/133] 0106.jpg
[23/133] 0109.jpg
[24/133] 011.jpg
[25/133] 0111.jpg
[26/133] 0112.jpg
[27/133] 0114.jpg
[28/133] 0115 2.jpg
[29/133] 0115.jpg
[30/133] 0116.jpg
[31/133] 0117.jpg
[32/133] 0119.jpg
[33/133] 0120.jpg
[34/133] 0121.jpg
[35/133] 0124 2.jpg
[36/133] 0124.jpg
[37/133] 0125.jpg
[38/133] 0126.jpg
[39/133] 0127.jpg
[40/133] 0128.jpg
[41/133] 0135.jpg
[42/133] 0136.jpg
[43/133] 0138.jpg
[44/133] 0143.jpg
[45/133] 0145.jpg
[46/133] 015_Kobiety z ziołami do święcenia - Bukowina Tatrzańska - 000836s.jpg
[47/133] 0161.jpg
[48/133] 0165.jpg
[49/133] 0171.jpg
[50/133] 0174

,file_path,prompt_en,period_label,score_period
0,inputs/0000.jpg,photograph taken in Poland up to 1944; interwa...,do 1944,0.536621
1,inputs/000202.jpg,photograph taken in Poland up to 1944; interwa...,do 1944,0.537109
2,inputs/000238.jpg,photograph taken in Poland up to 1944; interwa...,do 1944,0.536621
3,inputs/0003.jpg,photograph taken in Poland up to 1944; interwa...,do 1944,0.537109
4,inputs/000345.jpg,photograph taken in Poland up to 1944; interwa...,do 1944,0.537109


### 6. Zapis wyników (CSV + runlog)

In [7]:
"""
Zapis wyników zero-shot do CSV oraz wpis w runlog.jsonl.

Wejście:
- df_periods z kolumnami:
  'file_path', 'prompt_en', 'period_label', 'score_period'

Wyjście:
- plik CSV_PATH z pierwszą kolumną 'file_name'
- wpis w logs/runlog.jsonl z informacją o przebiegu
"""

df_periods = df_periods.copy()

# --- nowa kolumna: file_name i porządkowanie kolumn ---

df_periods["file_name"] = df_periods["file_path"].apply(lambda p: Path(p).name)

ordered_cols = (
    ["file_name"]
    + [c for c in df_periods.columns if c not in ["file_name"]]
)
df_periods = df_periods[ordered_cols]

# --- zapis CSV (bez kolumny timestamp) ---

df_periods.to_csv(CSV_PATH, index=False)
print("Zapisano wyniki do:", CSV_PATH)

# --- wpis do runlog.jsonl (timestamp tylko w runlogu) ---

timestamp = datetime.now(timezone.utc).isoformat()

runlog_entry = {
    "timestamp": timestamp,
    "step": "clip_zero_shot_periods",
    "n_images": int(len(df_periods)),
    "csv_path": str(CSV_PATH),
    "model_name": "ViT-B/32",
    "device": str(device),
    "dtype": str(IMG_DTYPE),
}

with RUNLOG_PATH.open("a", encoding="utf-8") as f:
    f.write(json.dumps(runlog_entry, ensure_ascii=False) + "\n")

print("Zaktualizowano runlog:", RUNLOG_PATH)

Zapisano wyniki do: /Users/olga/MetaLogic/outputs/csv/clip_zero_shot_subjects.csv
Zaktualizowano runlog: /Users/olga/MetaLogic/logs/runlog.jsonl


### 7. Streamlite in browser: Interaktywna klasyfikacja okresu historycznego

Aby uruchomić aplikację UI dla klasyfikacji okresów historycznych
(z poziomu terminala w katalogu projektu):

```bash
streamlit run apps/clip_zero_shot/app_clip_zero_shot.py

Ścieżka do katalogu aplikacji:  
/Users/olga/MetaLogic/apps/clip_zero_shot